
---

# **Notebook 10 : Traduction et Normalisation des Noms de Pays en Français avec ISO 3166**

---

### **1. Importation des Bibliothèques**

Dans cette section, nous importons les bibliothèques nécessaires pour la lecture des fichiers, la manipulation des données, et la traduction.



In [1]:
# Importation des bibliothèques
import os
import pandas as pd
from googletrans import Translator  # Assurez-vous d'avoir installé googletrans==4.0.0-rc1
import time
import re



### **2. Chargement de la Base ISO 3166 et Préparation des Données**

Nous allons charger la base ISO 3166 depuis `all.csv` et créer un dictionnaire de correspondance pour convertir les codes de pays en noms français normalisés.




In [2]:
# Chemin du fichier ISO
iso_path = 'note-books/phase_1_data/all.csv'
df_iso = pd.read_csv(iso_path)

# Création du dictionnaire de correspondance : code ISO (alpha-2 et alpha-3) et nom du pays en français
iso_mapping = {}

# Utilisation des codes alpha-2 et alpha-3 avec le nom du pays
for _, row in df_iso.iterrows():
    name = row['name']
    alpha_2 = row['alpha-2']
    alpha_3 = row['alpha-3']
    
    if isinstance(name, str):
        name = name.strip().capitalize()
        if isinstance(alpha_2, str):
            iso_mapping[alpha_2.strip().lower()] = name
        if isinstance(alpha_3, str):
            iso_mapping[alpha_3.strip().lower()] = name
        iso_mapping[name.lower()] = name  # Ajout du nom lui-même en minuscule pour la correspondance directe

# Vérification des premières correspondances pour confirmation
print("Exemple de mapping ISO:", list(iso_mapping.items())[:5])


Exemple de mapping ISO: [('af', 'Afghanistan'), ('afg', 'Afghanistan'), ('afghanistan', 'Afghanistan'), ('ax', 'Åland islands'), ('ala', 'Åland islands')]


---

### **3. Définition de la Fonction de Traduction et de Normalisation**

La fonction ci-dessous lit les chunks de fichiers, normalise les noms de pays, effectue les traductions, et les applique sur chaque chunk.


In [3]:
def traduire_pays_en_francais(input_directory, output_directory, batch_size=50, encoding='ISO-8859-1'):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    translator = Translator()
    pays_uniques = set()

    # Étape 1 : Collection des noms de pays uniques
    print("Étape 1 : Collection des noms de pays uniques")
    for file_name in os.listdir(input_directory):
        input_file_path = os.path.join(input_directory, file_name)
        
        # Gestion des erreurs d'encodage
        try:
            chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=10000, low_memory=False, on_bad_lines="skip", encoding=encoding)
        except UnicodeDecodeError:
            print(f"Problème d'encodage pour {file_name}. Réessai avec 'utf-8'.")
            chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=10000, low_memory=False, on_bad_lines="skip", encoding='utf-8')
        
        for chunk in chunk_iter:
            if 'countries' in chunk.columns:
                chunk['countries_cleaned'] = chunk['countries'].apply(
                    lambda x: [p.strip().removeprefix("en:") for p in str(x).split(',') if p.strip()] if pd.notna(x) else []
                )
                pays_uniques.update([p for sublist in chunk['countries_cleaned'] for p in sublist if re.match(r'^[a-zA-Z\s]+$', p)])

    # Étape 2 : Traduction des pays uniques en français
    print(f"Étape 2 : Traduction des {len(pays_uniques)} pays uniques")
    traduction_pays = {}
    pays_list = [p for p in pays_uniques if p]

    for i, country in enumerate(pays_list, start=1):
        try:
            # Nettoyage des préfixes "en:" avant la traduction
            clean_country = country.removeprefix("en:").strip()
            traduction = translator.translate(clean_country, src='auto', dest='fr').text
            traduction_pays[country] = traduction
            time.sleep(0.5)
            if i % batch_size == 0:
                print(f"{i} pays traduits sur {len(pays_list)}")
        except Exception as e:
            print(f"Erreur de traduction pour {country}: {e}")
            traduction_pays[country] = "unknown"

    # Étape 3 : Remplacement des noms de pays et sauvegarde des fichiers traduits
    print("Étape 3 : Remplacement des noms de pays et sauvegarde des fichiers traduits")
    for file_name in os.listdir(input_directory):
        input_file_path = os.path.join(input_directory, file_name)
        output_file_path = os.path.join(output_directory, file_name)

        try:
            chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=10000, low_memory=False, on_bad_lines="skip", encoding=encoding)
        except UnicodeDecodeError:
            print(f"Problème d'encodage pour {file_name}. Réessai avec 'utf-8'.")
            chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=10000, low_memory=False, on_bad_lines="skip", encoding='utf-8')
        
        for i, chunk in enumerate(chunk_iter):
            if 'countries' in chunk.columns:
                chunk['countries_cleaned'] = chunk['countries'].apply(
                    lambda x: [p.strip().removeprefix("en:") for p in str(x).split(',') if p.strip()] if pd.notna(x) else []
                )
                chunk['countries_translated'] = chunk['countries_cleaned'].apply(
                    lambda countries: ', '.join([traduction_pays.get(country, "unknown") for country in countries])
                )
                
                if i == 0:
                    chunk.to_csv(output_file_path, mode='w', index=False)
                else:
                    chunk.to_csv(output_file_path, mode='a', index=False, header=False)
        
        print(f"Fichier traduit et sauvegardé : {output_file_path}")

    print("Processus de traduction terminé.")


### **4. Répertoires d'Entrée et de Sortie**

Configurez les répertoires pour les chunks traduits.

In [4]:
# Répertoires d'entrée et de sortie
input_directory = 'note-books/phase_1_data/08_data_countries'
output_directory = 'note-books/phase_1_data/09_translated_countries'


---

### **5. Exécution de la Fonction de Traduction et Normalisation**

Exécutez la fonction `traduire_pays_en_francais` pour appliquer les traductions et normalisations sur les fichiers.

In [5]:
traduire_pays_en_francais(input_directory, output_directory)

Étape 1 : Collection des noms de pays uniques
Étape 2 : Traduction des 749 pays uniques
50 pays traduits sur 749
Erreur de traduction pour Filipinas: 'NoneType' object is not iterable
100 pays traduits sur 749
150 pays traduits sur 749
200 pays traduits sur 749
250 pays traduits sur 749
300 pays traduits sur 749
350 pays traduits sur 749
Erreur de traduction pour gr: 'NoneType' object is not iterable
400 pays traduits sur 749
Erreur de traduction pour my: 'NoneType' object is not iterable
450 pays traduits sur 749
500 pays traduits sur 749
Erreur de traduction pour International: 'NoneType' object is not iterable
550 pays traduits sur 749
600 pays traduits sur 749
650 pays traduits sur 749
Erreur de traduction pour ar: 'NoneType' object is not iterable
700 pays traduits sur 749
Erreur de traduction pour vu: 'NoneType' object is not iterable
Erreur de traduction pour GR: 'NoneType' object is not iterable
Étape 3 : Remplacement des noms de pays et sauvegarde des fichiers traduits
Fichier

---

### **Explications**

- **Normalisation ISO 3166** : Les noms de pays et codes ISO sont comparés à la base pour garantir des noms français standardisés.
- **Gestion des préfixes "en:"** : Tout préfixe "en:" est automatiquement supprimé.
- **Création de lignes séparées** : Chaque pays dans une même cellule est converti en une ligne distincte pour simplifier les analyses.

---